# cryoQuery

In [ ]:
#|default_exp cryo_query

In [1]:
# | export
import jupyter_black
import os
import polars as pl
import cryo

from dataclasses import dataclass

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")
jupyter_black.load()

In [2]:
cd ../

/home/evan/Documents/ethereum_block_explorer


In [8]:
# | export
@dataclass
class cryoQuery:
    streaming: bool = True

    def create_data_filepaths(self):
        """
        Creates folders for storing raw and combined data if they do not already exist.
        This function checks for the presence of 'data/raw'.
        If these directories do not exist, they are created. If they already exist,
        a message indicating their existence is printed.
        """
        raw_data_path = "data/raw"
        if not os.path.exists(raw_data_path):
            os.makedirs(raw_data_path)
            print("Data folder created.")
        else:
            print("Data folder already exists.")

    def query_blocks_txs(
        self,
        n_error_threshold: int = 1,
        retry_threshold: int = 5,
    ):
        """
        Fetches swap data from Uniswap V2 between specified block ranges.
        The function queries and retrieves swap data, attempting retries up to 'retry_threshold'
        if errors exceed 'n_error_threshold'. The retrieved data is saved as a Parquet file.

        :param start_block: The starting block number for the query (inclusive).
        :param end_block: The ending block number for the query (inclusive).
        :param n_error_threshold: The number of allowed errors before retrying the query.
        :param retry_threshold: The maximum number of retries for the query.
        """
        self.create_data_filepaths()
        n_errored = n_error_threshold + 1
        retry_count = 0

        # make cryo query
        while retry_count < retry_threshold:
            print(f"Retry count: {retry_count}")
            retry_count += 1
            if n_error_threshold < n_errored:
                output: dict[str] = cryo.freeze(
                    "blocks_and_transactions",
                    blocks=["latest"],
                    hex=True,
                    rpc="https://eth.merkle.io",
                    no_verbose=True,
                    output_dir="data/raw",
                    subdirs=["datatype"],
                )

                n_errored = output["n_errored"]
                print(f"Number of errors: {n_errored}")
            if n_errored == 0:
                print(f"{n_errored} == 0. Done!")
                break

In [9]:
cryo_query = cryoQuery()

In [10]:
cryo_query.query_blocks_txs()

Data folder already exists.
Data folder already exists.
Retry count: 0
cryo parameters
───────────────
- version: 0.3.0
- data: 
    - datatypes: blocks, transactions
    - block: 18,937,456
- source: 
    - network: ethereum
    - rpc url: https://eth.merkle.io
    - max requests per second: unlimited
    - max concurrent requests: unlimited
    - max concurrent chunks: 4
- output: 
    - chunk size: 1
    - chunks to collect: 1 / 1
    - output format: parquet
    - output dir: /home/evan/Documents/ethereum_block_explorer/data/raw
    - report file: $OUTPUT_DIR/.cryo/reports/2024-01-04_19-15-17.250415.json


schema for blocks
─────────────────
- block_number: uint32
- block_hash: hex
- timestamp: uint32
- author: hex
- gas_used: uint64
- extra_data: hex
- base_fee_per_gas: uint64
- chain_id: uint64

sorting blocks by: block_number

other available columns: parent_hash, state_root, transactions_root, receipts_root, logs_bloom, total_difficulty, size


schema for transactions
─────────

In [6]:
df

{'n_completed': 1, 'n_skipped': 0, 'n_errored': 0}